<a href="https://colab.research.google.com/github/leonardo-melniski/imersaodados-3/blob/main/imersaodados3_aula5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 5: Machine learning, sci-kit learning e desafios envolvidos.

Nesta aula, finalmente vamos criar o modelo de **Machine Learning** para classificar um novo composto com o intuito de prever se o MoA foi ativado ou não. Para isso, vamos usar a famosa biblioteca [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) para testar a **regressão logística** e entender o funcionamento de uma árvore de decisão (AD). Além disso, discutiremos problemas **importantes** como o **overfit**.



## Links extras bacanas

- Assista esse [vídeo](https://www.youtube.com/watch?v=K6HmoMnqeI8) para entender mais sobre o Overfit

- Quer entender como funciona o [Machine Learning](https://www.youtube.com/watch?v=CAu97npy7zQ)?

- Que tal conhecer como é trabalhar com [Machine Learning na Europa](https://www.youtube.com/watch?v=dFLrwi4Oz_o&t=1s)?

- [Otimização de hiperparâmetros](https://www.alura.com.br/artigos/otimizacao-de-parametros?utm_source=gnarus&utm_medium=timeline)